In [ ]:
pip install pandas matplotlib statsmodels prophet tensorflow


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
df = pd.read_csv(url)

# Parse dates and set as index
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

# Plot the dataset
plt.figure(figsize=(10, 6))
plt.plot(df['Passengers'], label='Airline Passengers')
plt.title('Monthly Airline Passenger Numbers')
plt.xlabel('Time')
plt.ylabel('Number of Passengers')
plt.legend()
plt.show()


In [ ]:
# Fit the ARIMA model
arima_model = ARIMA(df['Passengers'], order=(5, 1, 0))
arima_result = arima_model.fit()

# Forecast the next 12 months
arima_forecast = arima_result.forecast(steps=12)

# Plot the forecast
plt.figure(figsize=(10, 6))
plt.plot(df['Passengers'], label='Original')
plt.plot(arima_forecast, label='ARIMA Forecast', linestyle='--')
plt.title('ARIMA Forecast')
plt.legend()
plt.show()


In [ ]:
# Prepare data for Prophet
prophet_df = df.reset_index().rename(columns={'Month': 'ds', 'Passengers': 'y'})

# Fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(prophet_df)

# Make a forecast
future = prophet_model.make_future_dataframe(periods=12, freq='M')
prophet_forecast = prophet_model.predict(future)

# Plot the forecast
prophet_model.plot(prophet_forecast)
plt.title('Prophet Forecast')
plt.show()


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Passengers'].values.reshape(-1, 1))

# Prepare the data for LSTM
look_back = 12
X, y = [], []
for i in range(look_back, len(scaled_data)):
    X.append(scaled_data[i - look_back:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)

# Reshape for LSTM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build the LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm_model.fit(X, y, batch_size=32, epochs=10)

# Make predictions
lstm_predictions = lstm_model.predict(X)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

# Plot the predictions
plt.figure(figsize=(10, 6))
plt.plot(df['Passengers'], label='Original')
plt.plot(range(look_back, len(lstm_predictions) + look_back), lstm_predictions, label='LSTM Predictions', linestyle='--')
plt.title('LSTM Forecast')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error

# Compute RMSE for each model
arima_rmse = mean_squared_error(df['Passengers'][-12:], arima_forecast, squared=False)
prophet_rmse = mean_squared_error(df['Passengers'][-12:], prophet_forecast['yhat'][-12:], squared=False)
lstm_rmse = mean_squared_error(df['Passengers'][-len(lstm_predictions):], lstm_predictions.flatten(), squared=False)

print(f"ARIMA RMSE: {arima_rmse:.2f}")
print(f"Prophet RMSE: {prophet_rmse:.2f}")
print(f"LSTM RMSE: {lstm_rmse:.2f}")
